In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


# KERALA BUS DETAILS

In [4]:
# read the csv file
df=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_k.csv")
df

,Route_name,Route_link
0,Thiruvananthapuram to Kozhikode,https://www.redbus.in/bus-tickets/thiruvananth...
1,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
2,Kozhikode to Kottayam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
4,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
5,Thrissur to Kozhikode,https://www.redbus.in/bus-tickets/thrissur-to-...
6,Kozhikode to Kalpetta (kerala),https://www.redbus.in/bus-tickets/kozhikode-to...
7,Coimbatore to Ooty,https://www.redbus.in/bus-tickets/coimbatore-t...
8,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...
9,Thiruvananthapuram to Kozhikode,https://www.redbus.in/bus-tickets/thiruvananth...


In [11]:
# retrieve Bus details

In [3]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

     # Scroll to the bottom of the page to load all data
    last_height = driver_k.execute_script("return document.body.scrollHeight")
    while True:
        driver_k.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_k.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        

In [4]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_1 = pd.DataFrame(data)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_1.csv"
df_buses_1.to_csv(path,index=False)

# Andhrapradesh

In [9]:
df_1=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_AP.csv")
df_1

,Route_name,Route_link
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...
4,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
5,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
6,Anantapur (andhra pradesh) to Bangalore,https://www.redbus.in/bus-tickets/ananthapur-t...
7,Tirupati to Bangalore,https://www.redbus.in/bus-tickets/tirupathi-to...
8,Visakhapatnam to Vijayawada,https://www.redbus.in/bus-tickets/visakhapatna...
9,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...


In [12]:
#retrive the bus details
driver_AP = webdriver.Chrome()
Bus_names_AP = []
Bus_types_AP = []
Start_Time_AP = []
End_Time_AP = []
Ratings_AP = []
Total_Duration_AP = []
Prices_AP = []
Seats_Available_AP = []
Route_names = []
Route_links = []

for i,r in df_1.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_AP.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_AP.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_AP.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_AP.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_AP).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_AP.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

     # Scroll to the bottom of the page to load all data
    last_height = driver_AP.execute_script("return document.body.scrollHeight")
    while True:
        driver_AP.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_AP.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_AP.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_AP.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_AP.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_AP.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_AP.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_AP.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_AP.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_AP.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_AP.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_AP.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_AP.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_AP.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_AP.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_AP.append(ratings.text)
    for price_elem in price:
        Prices_AP.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_AP.append(seats_elem.text)
        




In [14]:
# from list to convert data frame
data_1 = {
    'Bus_name': Bus_names_AP,
    'Bus_type': Bus_types_AP,
    'Start_time': Start_Time_AP,
    'End_time': End_Time_AP,
    'Total_duration': Total_Duration_AP,
    'Price': Prices_AP,
    "Seats_Available":Seats_Available_AP,
    "Ratings":Ratings_AP,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_2 = pd.DataFrame(data_1)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_2.csv"
df_buses_2.to_csv(path,index=False)

# TELANGANA

In [2]:
df_3=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_T.csv")
df_3


,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [3]:
driver_T = webdriver.Chrome()
Bus_names_T = []
Bus_types_T = []
Start_Time_T = []
End_Time_T = []
Ratings_T = []
Total_Duration_T = []
Prices_T = []
Seats_Available_T = []
Route_names = []
Route_links = []

for i,r in df_3.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_T.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_T.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_T.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_T.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_T).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  # Adjust sleep time as needed
        
        new_page_source = driver_T.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_T.execute_script("return document.body.scrollHeight")
    while True:
        driver_T.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_T.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_T.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_T.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_T.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_T.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_T.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_T.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_T.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_T.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_T.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_T.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_T.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_T.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_T.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_T.append(ratings.text)
    for price_elem in price:
        Prices_T.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_T.append(seats_elem.text)

In [4]:
# from list to convert data frame
data_2 = {
    'Bus_name': Bus_names_T,
    'Bus_type': Bus_types_T,
    'Start_time': Start_Time_T,
    'End_time': End_Time_T,
    'Total_duration': Total_Duration_T,
    'Price': Prices_T,
    "Seats_Available":Seats_Available_T,
    "Ratings":Ratings_T,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_3 = pd.DataFrame(data_2)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_3.csv"
df_buses_3.to_csv(path,index=False)

# GOA

In [3]:
df_4=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_G.csv")
df_4

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
4,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
5,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
6,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
7,Goa to Pandharpur,https://www.redbus.in/bus-tickets/goa-to-pandh...
8,Solapur to Goa,https://www.redbus.in/bus-tickets/solapur-to-goa
9,Calangute (goa) to Goa Airport,https://www.redbus.in/bus-tickets/calangute-go...


In [4]:
# retrive the bus details
driver_G = webdriver.Chrome()
Bus_names_G = []
Bus_types_G = []
Start_Time_G = []
End_Time_G = []
Ratings_G = []
Total_Duration_G = []
Prices_G = []
Seats_Available_G = []
Route_names = []
Route_links = []

for i,r in df_4.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_G.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_G.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
   
    time.sleep(2)
    scrolling = True
    while scrolling:
        old_page_source = driver_G.page_source
        
        ActionChains(driver_G).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_G.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_G.execute_script("return document.body.scrollHeight")
    while True:
        driver_G.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_G.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_G.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_G.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_G.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_G.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_G.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_G.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_G.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_G.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_G.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_G.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_G.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_G.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_G.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_G.append(ratings.text)
    for price_elem in price:
        Prices_G.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_G.append(seats_elem.text)


In [5]:
# from list to convert data frame
data_3 = {
    'Bus_name': Bus_names_G,
    'Bus_type': Bus_types_G,
    'Start_time': Start_Time_G,
    'End_time': End_Time_G,
    'Total_duration': Total_Duration_G,
    'Price': Prices_G,
    "Seats_Available":Seats_Available_G,
    "Ratings":Ratings_G,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_4 = pd.DataFrame(data_3)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_4.csv"
df_buses_4.to_csv(path,index=False)

# RAJASTHAN

In [6]:
df_5=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_R.csv")
df_5

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [7]:
driver_R = webdriver.Chrome()
Bus_names_R= []
Bus_types_R = []
Start_Time_R = []
End_Time_R = []
Ratings_R = []
Total_Duration_R = []
Prices_R = []
Seats_Available_R = []
Route_names = []
Route_links = []

for i,r in df_5.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_R.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_R.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_R.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_R.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_R).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  # Adjust sleep time as needed
        
        new_page_source = driver_R.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_R.execute_script("return document.body.scrollHeight")
    while True:
        driver_R.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_R.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_R.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_R.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_R.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_R.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_R.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_R.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_R.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_R.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_R.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_R.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_R.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_R.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_R.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_R.append(ratings.text)
    for price_elem in price:
        Prices_R.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_R.append(seats_elem.text)

In [8]:
# from list to convert data frame
data_4 = {
    'Bus_name': Bus_names_R,
    'Bus_type': Bus_types_R,
    'Start_time': Start_Time_R,
    'End_time': End_Time_R,
    'Total_duration': Total_Duration_R,
    'Price': Prices_R,
    "Seats_Available":Seats_Available_R,
    "Ratings":Ratings_R,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_5 = pd.DataFrame(data_4)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_5.csv"
df_buses_5.to_csv(path,index=False)

# HARAYANA

In [32]:
df_6=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_H.csv")
df_6

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
2,Manali to Chandigarh,https://www.redbus.in/bus-tickets/manali-to-ch...
3,Chandigarh to Manali,https://www.redbus.in/bus-tickets/chandigarh-t...
4,Delhi to Manali,https://www.redbus.in/bus-tickets/delhi-to-manali
5,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
6,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
7,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
8,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi
9,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...


In [34]:
driver_H = webdriver.Chrome()
Bus_names_H= []
Bus_types_H = []
Start_Time_H = []
End_Time_H = []
Ratings_H = []
Total_Duration_H = []
Prices_H = []
Seats_Available_H = []
Route_names = []
Route_links = []

for i,r in df_6.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_H.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_H.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_H.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_H.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_H).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  # Adjust sleep time as needed
        
        new_page_source = driver_H.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_H.execute_script("return document.body.scrollHeight")
    while True:
        driver_H.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_H.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_H.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_H.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_H.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_H.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_H.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_H.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_H.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_H.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_H.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_H.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_H.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_H.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_H.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_H.append(ratings.text)
    for price_elem in price:
        Prices_H.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_H.append(seats_elem.text)

In [35]:
# from list to convert data frame
data_5 = {
    'Bus_name': Bus_names_H,
    'Bus_type': Bus_types_H,
    'Start_time': Start_Time_H,
    'End_time': End_Time_H,
    'Total_duration': Total_Duration_H,
    'Price': Prices_H,
    "Seats_Available":Seats_Available_H,
    "Ratings":Ratings_H,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_6 = pd.DataFrame(data_5)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_6.csv"
df_buses_6.to_csv(path,index=False)

# UTTARPRADESH

In [36]:
df_7=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_U.csv")
df_7

,Route_name,Route_link
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
1,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...
3,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
4,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
5,Badaun to Delhi,https://www.redbus.in/bus-tickets/badaun-to-delhi
6,Lucknow to Allahabad,https://www.redbus.in/bus-tickets/lucknow-to-a...
7,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...
8,Delhi to Badaun,https://www.redbus.in/bus-tickets/delhi-to-badaun
9,Agra to Bareilly,https://www.redbus.in/bus-tickets/agra-to-bare...


In [37]:
driver_U = webdriver.Chrome()
Bus_names_U= []
Bus_types_U = []
Start_Time_U = []
End_Time_U = []
Ratings_U = []
Total_Duration_U = []
Prices_U = []
Seats_Available_U = []
Route_names = []
Route_links = []

for i,r in df_7.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_U.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_U.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_U.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_U.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_U).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  # Adjust sleep time as needed
        
        new_page_source = driver_U.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_U.execute_script("return document.body.scrollHeight")
    while True:
        driver_U.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_U.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_U.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_U.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_U.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_U.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_U.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_U.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_U.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_U.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_U.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_U.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_U.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_U.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_U.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_U.append(ratings.text)
    for price_elem in price:
        Prices_U.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_U.append(seats_elem.text)

In [39]:
# from list to convert data frame
data_6 = {
    'Bus_name': Bus_names_U,
    'Bus_type': Bus_types_U,
    'Start_time': Start_Time_U,
    'End_time': End_Time_U,
    'Total_duration': Total_Duration_U,
    'Price': Prices_U,
    "Seats_Available":Seats_Available_U,
    "Ratings":Ratings_U,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_8 = pd.DataFrame(data_6)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_8.csv"
df_buses_8.to_csv(path,index=False)

# PUNJAB

In [41]:
df_8=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_P.csv")
df_8

,Route_name,Route_link
0,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
3,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
4,Phagwara to Delhi,https://www.redbus.in/bus-tickets/phagwara-to-...
5,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
6,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
7,Patiala to Delhi Airport,https://www.redbus.in/bus-tickets/patiala-to-d...
8,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...
9,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...


In [42]:
driver_P= webdriver.Chrome()
Bus_names_P= []
Bus_types_P = []
Start_Time_P = []
End_Time_P = []
Ratings_P= []
Total_Duration_P = []
Prices_P = []
Seats_Available_P = []
Route_names = []
Route_links = []

for i,r in df_8.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_P.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_P.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_P.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_P.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_P).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  # Adjust sleep time as needed
        
        new_page_source = driver_P.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_P.execute_script("return document.body.scrollHeight")
    while True:
        driver_P.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_P.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_P.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_P.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_P.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_P.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_P.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_P.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_P.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_P.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_P.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_P.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_P.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_P.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_P.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_P.append(ratings.text)
    for price_elem in price:
        Prices_P.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_P.append(seats_elem.text)

In [44]:
# from list to convert data frame
data_7 = {
    'Bus_name': Bus_names_P,
    'Bus_type': Bus_types_P,
    'Start_time': Start_Time_P,
    'End_time': End_Time_P,
    'Total_duration': Total_Duration_P,
    'Price': Prices_P,
    "Seats_Available":Seats_Available_P,
    "Ratings":Ratings_P,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_9 = pd.DataFrame(data_7)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_9.csv"
df_buses_9.to_csv(path,index=False)

# SOUTH BENGAL

In [9]:
df_9=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_S.csv")
df_9

,Route_name,Route_link
0,Durgapur to Calcutta,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Haldia to Calcutta,https://www.redbus.in/bus-tickets/haldia-to-ko...
3,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
6,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
7,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
8,Digha to Calcutta,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [13]:
#retrive the bus details
driver_S = webdriver.Chrome()
Bus_names_S = []
Bus_types_S = []
Start_Time_S = []
End_Time_S = []
Ratings_S = []
Total_Duration_S = []
Prices_S = []
Seats_Available_S = []
Route_names = []
Route_links = []

for i,r in df_9.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_S.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_S.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_S.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_S.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_S).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_S.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

     # Scroll to the bottom of the page to load all data
    last_height = driver_S.execute_script("return document.body.scrollHeight")
    while True:
        driver_S.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_S.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    

    # Extract bus details
    bus_name = driver_S.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_S.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_S.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_S.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_S.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_S.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_S.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_S.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_S.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_S.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_S.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_S.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_S.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_S.append(ratings.text)
    for price_elem in price:
        Prices_S.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_S.append(seats_elem.text)
        




In [14]:
# from list to convert data frame
data_9 = {
    'Bus_name': Bus_names_S,
    'Bus_type': Bus_types_S,
    'Start_time': Start_Time_S,
    'End_time': End_Time_S,
    'Total_duration': Total_Duration_S,
    'Price': Prices_S,
    "Seats_Available":Seats_Available_S,
    "Ratings":Ratings_S,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_10 = pd.DataFrame(data_9)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_10.csv"
df_buses_10.to_csv(path,index=False)

# Chandigar

In [8]:
df_12=pd.read_csv(r"C:\Users\phani\Downloads\capstone\redbus\df_C.csv")
df_12

,Route_name,Route_link
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
1,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
2,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
3,Chandigarh to Shimla,https://www.redbus.in/bus-tickets/chandigarh-t...
4,Chandigarh to Vrindavan,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
6,Chandigarh to Sujanpur (himachal pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
7,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
8,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
9,Vrindavan to Chandigarh,https://www.redbus.in/bus-tickets/vrindavan-to...


In [9]:
driver_C = webdriver.Chrome()
Bus_names_C = []
Bus_types_C = []
Start_Time_C = []
End_Time_C = []
Ratings_C = []
Total_Duration_C = []
Prices_C = []
Seats_Available_C = []
Route_names = []
Route_links = []

for i,r in df_12.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_C.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_C.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # click elements to views bus
    try:
        clicks = driver_C.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_C.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_C).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(2)  # Adjust sleep time as needed
        
        new_page_source = driver_C.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Scroll to the bottom of the page to load all data
    last_height = driver_C.execute_script("return document.body.scrollHeight")
    while True:
        driver_C.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait to load the page
        new_height = driver_C.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract bus details
    bus_name = driver_C.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_C.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_C.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_C.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_C.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_C.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_C.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_C.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_C.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_C.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_C.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_C.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_C.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_C.append(ratings.text)
    for price_elem in price:
        Prices_C.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_C.append(seats_elem.text)

In [10]:
# from list to convert data frame
data_10 = {
    'Bus_name': Bus_names_C,
    'Bus_type': Bus_types_C,
    'Start_time': Start_Time_C,
    'End_time': End_Time_C,
    'Total_duration': Total_Duration_C,
    'Price': Prices_C,
    "Seats_Available":Seats_Available_C,
    "Ratings":Ratings_C,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_11 = pd.DataFrame(data_10)
#convert dataframe to csv
path=r"C:\Users\phani\Downloads\capstone\redbus\df_buses_11.csv"
df_buses_11.to_csv(path,index=False)